In [0]:
import os
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator, image
import numpy as np

## For compile
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy

## Show history
from matplotlib import pyplot as plt


## Data Augmentation


In [0]:
!wget --no-check-certificate \
  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
  -O cats_and_dogs_filtered.zip

--2020-01-11 04:23:15--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 2607:f8b0:400e:c08::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘cats_and_dogs_filtered.zip’

cats_and_dogs_filte 100%[===================>]  65.43M   161MB/s    in 0.4s    

2020-01-11 04:23:16 (161 MB/s) - ‘cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [0]:
# ! unzip cats_and_dogs_filtered.zip

In [0]:
trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory = 'cats_and_dogs_filtered/train', target_size = (224, 224))

tsdata = ImageDataGenerator()
testdata = tsdata.flow_from_directory(directory = 'cats_and_dogs_filtered/validation', target_size = (224, 224))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


## Build Model


In [0]:

model = Sequential()

In [0]:
model = Sequential()

# 1 & 2 CNN layers
model.add(Conv2D(input_shape = (224,224,3), filters = 64, kernel_size = (3,3), padding="same", activation='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = "same", activation = "relu"))

model.add(MaxPool2D(pool_size = (2,2), strides = 2))

# 3 & 4 CNN layers
model.add(Conv2D(128, kernel_size = (3,3), padding = "same", activation="relu"))
model.add(Conv2D(128, kernel_size = (3,3), padding = "same", activation="relu"))

model.add(MaxPool2D(pool_size=(2,2), strides=2))

# 5, 6 & 7 CNN layers
model.add(Conv2D(256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(256, kernel_size=(3,3), padding="same", activation="relu"))

model.add(MaxPool2D(pool_size=(2,2), strides=2))

# 8, 9, & 10 CNN layers
model.add(Conv2D(512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(512, kernel_size=(3,3), padding="same", activation="relu"))

model.add(MaxPool2D(pool_size=(2,2), strides=2))

# 11, 12 & 13 CNN Layers
model.add(Conv2D(512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(512, kernel_size=(3,3), padding="same", activation="relu"))

model.add(MaxPool2D(pool_size=(2,2), strides=2))

# 14, 15 & 16 CNN layers
model.add(Flatten())
model.add(Dense(units=4096, activation="relu"))
model.add(Dense(units=4096, activation="relu"))

# output layer
model.add(Dense(units=2, activation="softmax"))

## Compile model and fit the data


In [0]:
adam = Adam(lr=0.001)

model.compile(optimizer=adam, loss = categorical_crossentropy, metrics=['accuracy'])

### Callback methods



In [0]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("catvsdog.h5", monitor='val_acc', save_best_only=True, save_weights_only=False, mode = 'auto', period=1)

early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, mode='auto',)


In [0]:
hist = model.fit_generator(steps_per_epoch=100, generator=traindata, validation_data=testdata, validation_steps=10, epochs=100, 
                           callbacks=[checkpoint, early])

Epoch 1/100





 88/100 [=========================>....] - ETA: 11:56 - loss: 7.9523 - acc: 0.5018

## Model Evaluate

In [0]:


plt.plot(hist.history["acc"])
plt.plot(hist.history["val_acc"])
plt.plot(hist.history["loss"])
plt.plot(hist.history["val_loss"])
plt.title("Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy","Loss","Validation Loss"])
plt.show()

## Prediction

In [0]:
## Load the test image

img = image.load_img("test.jpeg",target_size=(224,224))
img = np.asarray(img)
plt.imshow(img)
img = np.expand_dims(img, axis=0)

## load saved model

model = load_model("catvsdog.h5")
prediction = model.predict(img)

if prediction[0][0] > prediction[0][1]:
  print("Cat")
else:
  print("Dog")